In [1]:
import time
from time import sleep
import json
import ccxt
import requests
import datetime
import pandas as pd
import numpy as np
from scipy import optimize
from IPython.display import display

In [2]:
def getCandlestick(number, period):
        """
        number:ローソク足の数．period:ローソク足の期間（文字列で秒数を指定，Ex:1分足なら"60"）．cryptowatchはときどきおか
        しなデータ（price=0）が含まれるのでそれを除く．
        """

        now = datetime.datetime.now().strftime('%s') # 現在時刻の取得
        r = requests.get('https://testnet.bitmex.com/api/udf/history?symbol=XBTUSD&resolution=' + str(int(period)/60) + '&from=' +
        str(int(now)-int(period)*(number-1)) + '&to=' + now)
        ohlcv = r.json()
        df_ = pd.DataFrame(ohlcv)
        df_ = df_.rename(columns = {"c":"close", "h":"high", "l":"low", "o":"open", "t":"index", "v":"value"})
        df_ = df_[["index", "close", "low",  "high", "open", "s", "value"]]

        time_date = []
        for i in df_.iloc[:,0]:
            time = datetime.datetime.fromtimestamp(i)
            time_date.append(time)
        dti = pd.DatetimeIndex(time_date)
        df_["index"] = dti
        df_ = df_.set_index("index")
        df_ = df_.reset_index()
        #df_ = df_.rename(columns={'t': 'date_date'})
        df_ = df_.rename(columns={'index': 'date_date'})
        return df_

In [3]:
def psar(barsdata):

        length = len(barsdata)
        high = list(barsdata['high'])
        low = list(barsdata['low'])
        close = list(barsdata['close'])
        psar = close[0:len(close)]
        psarbull = [None] * length
        psarbear = [None] * length
        bull = True

        iaf = 0.01
        maxaf = 0.2

        af = iaf
        hp = high[0]
        lp = low[0]

        for i in range(2,length):
            if bull:
                psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
            else:
                psar[i] = psar[i - 1] + af * (lp - psar[i - 1])

            reverse = False

            if bull:
                if low[i] < psar[i]:
                    bull = False
                    reverse = True
                    psar[i] = hp
                    lp = low[i]
                    af = iaf
            else:
                if high[i] > psar[i]:
                    bull = True
                    reverse = True
                    psar[i] = lp
                    hp = high[i]
                    af = iaf

            if not reverse:
                if bull:
                    if high[i] > hp:
                        hp = high[i]
                        af = min(af + iaf, maxaf)
                    if low[i - 1] < psar[i]:
                        psar[i] = low[i - 1]
                    if low[i - 2] < psar[i]:
                        psar[i] = low[i - 2]
                else:
                    if low[i] < lp:
                        lp = low[i]
                        af = min(af + iaf, maxaf)
                    if high[i - 1] > psar[i]:
                        psar[i] = high[i - 1]
                    if high[i - 2] > psar[i]:
                        psar[i] = high[i - 2]

            if bull:
                psarbull[i] = psar[i]
            else:
                psarbear[i] = psar[i]
        
        barsdata["psarbear"] = psarbear
        barsdata["psarbull"] = psarbull
        
        return barsdata


In [4]:
df_ =  getCandlestick(1200, "300")
result = psar(df_)
result = result.fillna(0)
result = result[10:len(result)]
result

,date_date,close,low,high,open,s,value,psarbear,psarbull
10,2018-05-19 19:25:00,8210.5,8206.0,8246.0,8230.0,ok,1001079,0.000000,8169.115000
11,2018-05-19 19:30:00,8214.5,8210.5,8214.5,8210.5,ok,54226,0.000000,8170.652700
12,2018-05-19 19:35:00,8215.0,8209.5,8218.0,8214.5,ok,193108,0.000000,8172.159646
13,2018-05-19 19:40:00,8226.5,8185.5,8250.0,8215.0,ok,4005476,0.000000,8173.636453
14,2018-05-19 19:45:00,8231.5,8226.0,8244.5,8226.5,ok,264449,0.000000,8175.927359
15,2018-05-19 19:50:00,8231.5,8218.5,8233.5,8231.5,ok,115388,0.000000,8178.149539
16,2018-05-19 19:55:00,8234.5,8232.0,8243.5,8231.5,ok,51688,0.000000,8180.305053
17,2018-05-19 20:00:00,8236.0,8200.5,8241.0,8234.5,ok,859729,0.000000,8182.395901
18,2018-05-19 20:05:00,8223.0,8219.5,8237.5,8236.0,ok,113101,0.000000,8184.424024
19,2018-05-19 20:10:00,8227.5,8222.5,8228.5,8223.0,ok,22541,0.000000,8186.391303


In [5]:
bearin = pd.DataFrame()
for i in range(len(result)):
    if result.iloc[i]["psarbear"] != 0:
        if result.iloc[i-1]["psarbear"] == 0:
            bearin = bearin.append(result.iloc[i])

In [6]:
bearin

,close,date_date,high,low,open,psarbear,psarbull,s,value
31,8247.5,2018-05-19 21:10:00,8307.5,8207.5,8299.5,8300.0,0.0,ok,3507272.0
65,8260.5,2018-05-20 00:00:00,8282.0,8225.5,8263.0,8303.0,0.0,ok,1089070.0
98,8250.0,2018-05-20 02:45:00,8274.5,8246.0,8264.5,8305.0,0.0,ok,811885.0
137,8176.0,2018-05-20 06:00:00,8249.0,8171.0,8237.0,8256.5,0.0,ok,3562399.0
150,8204.0,2018-05-20 07:05:00,8254.0,8154.0,8247.0,8255.5,0.0,ok,10810527.0
222,8274.5,2018-05-20 13:05:00,8275.0,8261.0,8262.0,8277.0,0.0,ok,98417.0
224,8271.0,2018-05-20 13:15:00,8277.0,8251.0,8276.0,8277.0,0.0,ok,446305.0
274,8273.5,2018-05-20 17:25:00,8275.5,8246.0,8273.0,8277.0,0.0,ok,681151.0
305,8261.0,2018-05-20 20:00:00,8301.5,8210.0,8295.5,8322.0,0.0,ok,3981427.0
380,8437.0,2018-05-21 02:15:00,8451.5,8427.5,8449.0,8565.5,0.0,ok,704664.0


In [7]:
bullin = pd.DataFrame()
for i in range(len(result)):
    if result.iloc[i]["psarbull"] != 0:
        if result.iloc[i-1]["psarbull"] == 0:
            bullin = bullin.append(result.iloc[i])

In [8]:
bullin

,close,date_date,high,low,open,psarbear,psarbull,s,value
10,8210.5,2018-05-19 19:25:00,8246.0,8206.0,8230.0,0.0,8169.115,ok,1001079.0
49,8293.5,2018-05-19 22:40:00,8301.0,8267.0,8266.0,0.0,8207.500,ok,641492.0
73,8291.5,2018-05-20 00:40:00,8300.0,8291.5,8291.5,0.0,8225.500,ok,1545350.0
135,8240.5,2018-05-20 05:50:00,8256.5,8237.0,8239.0,0.0,8200.000,ok,1364763.0
149,8247.0,2018-05-20 07:00:00,8255.5,8189.5,8225.0,0.0,8171.000,ok,6201374.0
177,8212.5,2018-05-20 09:20:00,8214.0,8210.0,8210.0,0.0,8130.500,ok,40640.0
223,8276.0,2018-05-20 13:10:00,8277.0,8269.0,8274.5,0.0,8261.000,ok,129969.0
267,8262.0,2018-05-20 16:50:00,8268.0,8259.0,8264.0,0.0,8250.500,ok,68018.0
285,8279.5,2018-05-20 18:20:00,8279.5,8200.0,8236.0,0.0,8210.000,ok,5019042.0
315,8296.0,2018-05-20 20:50:00,8300.0,8290.0,8292.0,0.0,8195.000,ok,297947.0


In [9]:
if (bearin.index[0] < bullin.index[0]) & (bearin.index[-1] > bullin.index[-1]):
    short = bearin["close"].sum() - bullin["close"].sum() - float(bearin.tail(1)["close"])
    long = bearin["close"].sum() - bullin["close"].sum() - float(bearin.head(1)["close"])
    print(1)
    print(short)
    print(long)
    profit = short + long
    
if (bearin.index[0] > bullin.index[0]) & (bearin.index[-1] > bullin.index[-1]):
    short = bearin["close"].sum() - bullin["close"].sum() - float(bearin.tail(1)["close"]) + float(bullin.head(1)["close"])
    long = bearin["close"].sum() - bullin["close"].sum() 
    print(2)
    print(short)
    print(long)
    profit = short + long
    
if (bearin.index[0] < bullin.index[0]) & (bearin.index[-1] < bullin.index[-1]):
    short = bearin["close"].sum() - bullin["close"].sum() 
    long = bearin["close"].sum() - bullin["close"].sum() - float(bearin.head(1)["close"]) + float(bullin.tail(1)["close"])
    print(3)
    print(short)
    print(long)
    profit = short + long
    
if (bearin.index[0] > bullin.index[0]) & (bearin.index[-1] < bullin.index[-1]):
    short = bearin["close"].sum() - bullin["close"].sum() + float(bullin.head(1)["close"])
    long = bearin["close"].sum() - bullin["close"].sum() + float(bullin.tail(1)["close"])
    print(4)
    print(short)
    print(long)
    profit = short + long

2
-557.0
-787.5


In [10]:
profit

-1344.5

In [11]:
bullin.index[0]

10